In [133]:
import pandas as pd
import numpy as np
import talib as ta
import os
import matplotlib.pyplot as plt
import warnings
import pymongo
from datetime import datetime
warnings.filterwarnings('ignore')

In [134]:
dataCoin = pd.read_csv('../data/dataCoin.csv')
dataCoin.set_index('datetime',inplace=True)
dataCoins = []
dataCoins.append(dataCoin.loc['2018-05-01':'2019-05-01'])
dataCoins.append(dataCoin.loc['2019-05-01':'2020-05-01'])
dataCoins.append(dataCoin.loc['2020-05-01':'2021-05-01'])
dataCoins.append(dataCoin.loc['2020-05-01':'2021-09-23'])
dataCoins.append(dataCoin.loc['2021-05-01':'2021-09-23'])
dataCoins.append(dataCoin.loc[:])

In [135]:
def cal_indicator(dataCoin):    
    dataCoin['pctChange'] = dataCoin['close'].pct_change()
    dataCoin['pctStd'] = ta.STDDEV(dataCoin['pctChange'], 24*5)*(24**0.5)
    dataCoin['pctStdVma'] = ta.SUM(dataCoin['pctStd']*dataCoin['volume'], 24*50) / ta.SUM(dataCoin['volume'], 24*50)
    dataCoin['closeVma'] = ta.SUM(dataCoin['close']*dataCoin['volume'], 24*50) / ta.SUM(dataCoin['volume'], 24*50)
#     dataCoin['pctStdVSma'] = ta.EMA(dataCoin.pctStdVma, 24*3)
#     dataCoin['pctStdVLma'] = ta.MA(dataCoin.pctStdVma, 24*10)
for data in dataCoins:
    cal_indicator(data)

In [136]:
def get_stat(df):
    se = df.describe()['pctStd']
    se = se.append(pd.Series(df.skew()['pctStd'],index=['skew']))
    se = se.append(pd.Series(df.kurt()['pctStd'],index=['kurt']))
    se = se.append(pd.Series(se['75%']-se['25%'],index=['75%-25%']))
    return se

In [137]:
ses = []
for data in dataCoins:
    ses.append(get_stat(data))
df = pd.DataFrame(ses)
df = df.T
df.columns = ['2018.5 - 2019.5',
              '2019.5 - 2020.5',
              '2020.5 - 2021.5',
              '2020.5 - 2021.9',
              '2021.5 - 2021.9',
              '2018.5 - 2021.9']
df

,2018.5 - 2019.5,2019.5 - 2020.5,2020.5 - 2021.5,2020.5 - 2021.9,2021.5 - 2021.9,2018.5 - 2021.9
count,8606.000000,8635.000000,8625.000000,12077.000000,3332.000000,29558.000000
mean,0.044842,0.042861,0.043741,0.047588,0.057621,0.045311
std,0.019909,0.024184,0.019275,0.021991,0.025500,0.022128
min,0.012783,0.015876,0.012339,0.012339,0.026804,0.012339
25%,0.029456,0.029858,0.030793,0.033405,0.042123,0.031018
50%,0.040496,0.037355,0.038087,0.042010,0.049184,0.040284
75%,0.057324,0.048837,0.053503,0.056531,0.062675,0.054527
max,0.109565,0.202287,0.113070,0.162451,0.162451,0.202287
skew,0.765256,3.449153,1.193070,1.728748,2.123655,2.160471
kurt,0.213007,16.875474,1.167480,4.446964,4.724682,8.604505


In [138]:
data_over_ma = []
data_below_ma = []
dataCoinsMA = []
dataCoinsMA.append(dataCoin.loc['2018-05-01':'2020-05-01'])
dataCoinsMA.append(dataCoin.loc['2020-05-01':'2020-09-23'])
for data in dataCoinsMA:
    cal_indicator(data)

for data in dataCoinsMA:
    data_over_ma.append(data[data['close'] >= data['closeVma']])
    data_below_ma.append(data[data['close'] < data['closeVma']])

In [139]:
ses = []
for data in data_over_ma:
    ses.append(get_stat(data))
df = pd.DataFrame(ses)
df = df.T
df.columns = ['2018.5 - 2020.5',
              '2020.5 - 2021.9']
print('OVER MA')
df

OVER MA


,2018.5 - 2020.5,2020.5 - 2021.9
count,6546.000000,1781.000000
mean,0.042114,0.033261
std,0.017615,0.012764
min,0.015050,0.012853
25%,0.030393,0.023907
50%,0.037939,0.031803
75%,0.050887,0.038916
max,0.109565,0.066950
skew,1.242668,0.808884
kurt,1.821309,0.325186


In [140]:
ses = []
for data in data_below_ma:
    ses.append(get_stat(data))
df = pd.DataFrame(ses)
df = df.T
df.columns = ['2018.5 - 2020.5',
              '2020.5 - 2021.9']
print('BELOW MA')
df

BELOW MA


,2018.5 - 2020.5,2020.5 - 2021.9
count,9736.000000,497.000000
mean,0.044095,0.046254
std,0.025490,0.022500
min,0.012783,0.012339
25%,0.028027,0.032949
50%,0.036376,0.042016
75%,0.054454,0.068889
max,0.202287,0.082944
skew,2.647676,0.305538
kurt,11.213942,-1.031585
